# 環境設定


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
soundlist = os.listdir('/content/drive/My Drive/test_sound')
soundlist

In [ ]:
pip install pydub librosa

In [8]:
from pydub.silence import split_on_silence
import os
import librosa
import wave
import numpy as np
import pylab as plt
from pydub import AudioSegment

# 音檔靜音切割方法

In [44]:
class mute_cut:
  # 依照聲音檔中的靜音來做切割

  def __init__(self, path= "" ):
      self.isfile = False
      self.have_wav = False
      self.path = path
      if os.path.isfile(path ):
        # 如果是檔案路徑
        # 將檔案讀近來來做切割
        self.isfile = True
        if ".wav" in path:
          self.data = self.sound_split( path )
          self.onename = (path.split("/")[-1]).split(".")[0]
          self.have_wav = True
        else:
          print("此路徑的檔案不是wav檔")
      elif os.path.isdir(path ):
        # 如果是資料夾路徑
        # 紀錄底下所有資料名稱
        self.filename=[ x for x in os.listdir(path) if ".wav" in x]
        if len(self.filename) == 0:
          print("此資料夾沒有wav檔")
        else:
          self.have_wav = True
      

  def sound_split(self,filepath):
      sound = AudioSegment.from_wav(filepath)
      dBFS = sound.dBFS # 指定最大聲音
      # print(dBFS)
      chunks = split_on_silence(sound, 
          min_silence_len = 1000,
          silence_thresh = dBFS-16)
      return chunks

  def output_cutting(self,path_to=""):
      if not self.have_wav :
        print("沒有正確的資料進行分割")
        return None
      if path_to == "":
        path_to = "."
      elif path_to != "" and not os.path.exists(path_to): 
        # 判斷目錄是否存在，不存在就建立檔案
        os.makedirs(path_to)
        
      if self.isfile :
        # 檔案路徑處理
        for i in range(len(self.data)):
          self.data[i].export( f"{path_to}/{self.onename}_{i}.wav", format="wav")

      else:
        # 資夾路徑的處理
        for i in range(len(self.filename)):
          data = self.sound_split( self.path+"/"+self.filename[i])
          name = (self.filename[i].split("/")[-1]).split(".")[0]
          for j in range(len(data)):
            data[j].export( f"{path_to}/{name}_{j}.wav", format="wav")



> 測試


In [45]:
temp  = mute_cut("/content/drive/My Drive/test_sound/貓(喵)老大打架_.wav")
temp.output_cutting("/content/drive/My Drive/test_sound/test")

# 音檔固定長度切割

In [50]:

def create_data(path_from,path_to,item_len = 2000):
    if path_from.endswith('.wav'):# 輸入的資料來源如果是.wav
        files = [path_from]
    else: # 輸入的資料來源如果是資料夾
        files = os.listdir(path_from)
        files = [f"{path_from}/{ f}" for f in files if f.endswith('.wav')]
    if  not os.path.exists(path_to): # 判斷目錄是否存在，不存在就建立檔案
        os.mkdir(path_to)
    
    for file in files:
        file_name = "".join((file.split('/').pop()).split("_")[:3])
        file_name = file_name.split('.')[0]
        audio = AudioSegment.from_file(file)
        audio_len = len(audio)

        steps = audio_len // item_len+1
        for step in range(steps):
            item_audio = audio[step * item_len:( (step + 1) * item_len if (step + 1) * item_len < audio_len else audio_len )]
            save_audio = item_audio + AudioSegment.silent(item_len - len(item_audio))
            new_path = f"{path_to}/{file_name}_{step}.wav"
            save_audio.export( new_path)
    
    

>測試

In [51]:
path_from = "/content/drive/My Drive/test_sound/貓(喵)老大打架_.wav"
path_to = "/content/drive/My Drive/test_sound/test2"
create_data(path_from , path_to)